In [75]:
import numpy as np
from numpy import ma
import pandas as pd
from scipy.stats import norm

np.set_printoptions(precision=5, suppress=True)

# Set random seed to student number
np.random.seed(46387334) # TODO: UNCOMMENT

In [76]:
# Helper functions
def sig_fig(X, sigfigs):
    exp = np.floor(ma.log10(abs(X)).filled(0))
    return np.round(X*10**-exp, sigfigs-1) * 10**exp

def get_diff(a1, a2):
    print(f"{a1.shape=}")
    print(f"{a2.shape=}")

    rows, cols = a1.shape
    for i in range(rows):
        for j in range(cols):
            print(f"[{i},{j}]: {round(a1[i, j], 3) :>5} vs {round(a2[i, j], 3) :>7}  |  Error: {(a1[i, j] - a2[i, j]) * (100 / a1[i, j])  :.2f}%")

## 1.a)

### Part 1. Generate 100 observations using only U[0, 1]

In [77]:
# Variables that make up multivariate normal distribution
vars_h = [
    'logcp',
    'ejection_fraction',
    'sqrtplat',
    'recipsc',
    'serum_sodium'
]
n_vars = len(vars_h)

# mu_h hat - Mean vector for multivariate normal distribution
mu_h = np.array([
    [5.66 ],
    [38.1 ],
    [505  ],
    [0.861],
    [137  ]
])

# Sigma_h hat - Covariance matrix for multivariate normal distribution
sigma_h = np.array([
    [1.29,   -0.928, 1.01,   0.0235, 0.0953],
    [-0.928, 140,    77.8,   0.514,  9.19  ],
    [1.01,   77.8,   8757,   1.82,   23.0  ],
    [0.0235, 0.514,  1.82,   0.100,  0.354 ],
    [0.0953, 9.19,   23.0,   0.354,  19.5  ]
])


# Number of observations to generate
n_obs = 100

# 1. Generate multivariate uniform observations from U[0, 1]
# i.e. Multivariate Uniform disribution with the same dimentions as our multivariate normal
obs_uniform = np.random.uniform(low=0, high=1, size=(n_vars, n_obs))

# 2. Convert uniform observations to multivariate standard normal observations i.e. N(0, I) 
# norm.ppf comuptes the inverse of the cdf
obs_std_norm = norm.ppf(obs_uniform)

# 3. Convert standard normal observations to our multivariate normal
# 3.1. Determine decomposition of the covariance matrix
A_h = np.linalg.cholesky(sigma_h)

# 3.2. Convert observations
obs_h = mu_h + A_h @ obs_std_norm

# Save the data to a csv
# Transpose the data to ensure that each row represents a sample
df = pd.DataFrame(obs_h.T, columns=vars_h)
df.to_csv('question_1_a_obs.csv', index=False)

### Part 2. Estimate parameters based on observations

In [78]:
# Estimate the mean and cov based on the observations
est_mu_h = np.mean(obs_h, axis=1, keepdims=True)
print("Estimated Mu:")
print(sig_fig(est_mu_h, 5))

est_sigma_h = np.cov(obs_h)
print("\nEstimated Sigma:")
print(sig_fig(est_sigma_h, 5))

est_corr = np.corrcoef(obs_h)
print("\nEstimated Correlation:")
print(sig_fig(est_sigma_h, 5))

Estimated Mu:
[[  5.5101 ]
 [ 39.127  ]
 [506.65   ]
 [  0.84898]
 [136.5    ]]

Estimated Sigma:
[[   1.1266    -1.4698     5.8935    -0.02918   -0.0236 ]
 [  -1.4698   150.84     -38.346      1.0062    16.898  ]
 [   5.8935   -38.346   9725.3        0.34479   30.115  ]
 [  -0.02918    1.0062     0.34479    0.07184    0.2428 ]
 [  -0.0236    16.898     30.115      0.2428    18.044  ]]

Estimated Correlation:
[[   1.1266    -1.4698     5.8935    -0.02918   -0.0236 ]
 [  -1.4698   150.84     -38.346      1.0062    16.898  ]
 [   5.8935   -38.346   9725.3        0.34479   30.115  ]
 [  -0.02918    1.0062     0.34479    0.07184    0.2428 ]
 [  -0.0236    16.898     30.115      0.2428    18.044  ]]
